# Description

Over 20k recipes listed by recipe rating, nutritional information and assigned category (sparse). I may later upload a version binned by recipe creation date and also including recipe ingredients.

Use the 'full_format_recipes.json' file to interact with all recipe data, 'epi_r.csv' drops ingredients and directions in favour of sparse category dummies.

In [1]:
import polars as pl
import numpy as np
import kagglehub
import shutil
import pandas as pd
import re

pl.Config.set_fmt_str_lengths(100)  # Ajuste o tamanho conforme necessário


c:\Users\fabia\Downloads\recipes_db\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


polars.config.Config

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("irkaal/foodcom-recipes-and-reviews")

print("Path to dataset files:", path)

100%|██████████| 723M/723M [00:32<00:00, 23.7MB/s] 

Extracting files...


Path to dataset files: C:\Users\fabia\.cache\kagglehub\datasets\irkaal\foodcom-recipes-and-reviews\versions\2


In [4]:

# # Download latest version
# path = kagglehub.dataset_download("irkaal/foodcom-recipes-and-reviews")

# print("Path to dataset files:", path)
# # Specify the source file and destination path
# source = path
# destination = r"C:\Users\fabia\Downloads\recipes_db\data_raw"

# # Move the file
# try:
#     shutil.move(source, destination)
#     print(f"File moved to {destination}")
# except FileNotFoundError:
#     print("The source file does not exist.")
# except PermissionError:
#     print("You do not have permission to move the file.")
# except Exception as e:
#     print(f"An error occurred: {e}")

In [73]:
# read files with polars
file_path_csv = r"C:\Users\fabia\Downloads\recipes_db\data_raw\2\epi_r.csv"
file_path_json = r"C:\Users\fabia\Downloads\recipes_db\data_raw\2\full_format_recipes.json"
df_json = pl.read_json(file_path_json)
df = pl.read_csv(file_path_csv)



TypeError: `schema_overrides` should be of type list or dict

Tabela Receitas (informações gerais sobre cada receita)
- Colunas principais:
<br>id (chave primária, única para cada receita)</br>
<br>nome (nome da receita)</br>
<br>tempo_preparo (tempo necessário em minutos)</br>
<br>calorias (valor calórico por porção)</br>
<br>sodium (quantidade de sódio por porção)</br>
<br>rating (nota de avaliação)
<br>categoria (tipo de dieta ou categoria geral)</br>

In [9]:
# Filter important columns for specics diets our recipes categories
columns = ['title', 'rating', 'calories', 'protein', 'fat', 'sodium', '22-minute meals', '3-ingredient recipes', '30 days of groceries',
           'advance prep required', 'dairy', 'dairy free', 'egg','digestif','dinner', 'fat free', 'fish', 'grill', 'healthy','low cal',
           'low carb', 'low cholesterol', 'low fat','low sodium','low sugar','lunch', 'meat','no sugar added','no-cook','pork', 'quick & easy',
           'quick and healthy','soy','soy free','vegan','vegetarian'
           ]

recipes = df[columns]
recipes.head()

title,rating,calories,protein,fat,sodium,22-minute meals,3-ingredient recipes,30 days of groceries,advance prep required,dairy,dairy free,egg,digestif,dinner,fat free,fish,grill,healthy,low cal,low carb,low cholesterol,low fat,low sodium,low sugar,lunch,meat,no sugar added,no-cook,pork,quick & easy,quick and healthy,soy,soy free,vegan,vegetarian
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Lentil, Apple, and Turkey Wrap """,2.5,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Boudin Blanc Terrine with Red Onion Confit """,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Potato and Fennel Soup Hodge """,3.75,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Mahi-Mahi in Tomato Olive Sauce """,5.0,null,null,null,null,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"""Spinach Noodle Casserole """,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
recipes.columns

In [10]:
# Rename columns before concat

rename_columns_df1 = {'title': 'recipeTitle',
                    'rating': 'rating',
                    'calories': 'calories',
                    'protein': 'proteinGrams',
                    'fat': 'fatGrams',
                    'sodium': 'sodiumMg',
                    '22-minute meals': 'twentyTwoMinuteMeals',
                    '3-ingredient recipes': 'threeIngredientRecipes',
                    '30 days of groceries': 'thirtyDaysOfGroceries',
                    'advance prep required': 'advancePrepRequired',
                    'dairy free': 'dairyFree',
                    'fat free': 'fatFree',
                    'low cal': 'lowCal',
                    'low carb': 'lowCarb', 
                    'low cholesterol': 'lowCholesterol',
                    'low fat': 'lowFat',
                    'low sodium': 'lowSodium',
                    'low sugar': 'lowSugar',
                    'no sugar added': 'zeroSugar',
                    'no-cook': 'noCook',
                    'quick & easy': 'quickEasy',
                    'quick and healthy': 'quickHealthy',
                    'soy free': 'soyFree'
                  }

recipes = recipes.rename(rename_columns_df1)

rename_columns_df2 = {'title': 'recipeTitle',
                    'rating': 'rating',
                    'calories': 'calories',
                    'protein': 'proteinGrams',
                    'fat': 'fatGrams',
                    'sodium': 'sodiumMg',
                    'directions': 'directions',
                    'desc': 'desc',
                    'ingredients': 'ingredients',
                  }

df_json = df_json.rename(rename_columns_df2)
df_json.head()


directions,fatGrams,date,categories,calories,desc,proteinGrams,rating,recipeTitle,ingredients,sodiumMg
list[str],f64,str,list[str],f64,str,f64,f64,str,list[str],f64
"[""1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool."", ""2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper."", ""3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.""]",7.0,"""2006-09-01T04:00:00.000Z""","[""Sandwich"", ""Bean"", … ""Cookie""]",426.0,null,30.0,2.5,"""Lentil, Apple, and Turkey Wrap ""","[""4 cups low-sodium vegetable or chicken stock"", ""1 cup dried brown lentils"", … ""1/2 head Bibb lettuce""]",559.0
"[""Combine first 9 ingredients in heavy medium saucepan. Add 3 shallots. Bring to simmer. Remove from heat, cover and let stand 30 minutes. Chill overnight."", ""Preheat oven to 325°F. Line 7-cup pâté or bread pan with plastic wrap. Melt butter in heavy small skillet over low heat. Add remaining 5 shallots. Cover and cook until very soft, stirring occasionally, about 15 minutes. Transfer to processor. Add pork, eggs, flour and Port and puree. Strain cream mixture, pressing on solids to extract as much liquid as possible. With processor running, add cream through feed tube and process just until combined with pork. Transfer to large bowl. Mix in currants."", … ""Heat oil in heavy large skillet over medium-high heat. Add onions and sauté until crisp-tender, about 8 minutes. Add all remaining ingredients and stir until reduced to thick glaze, about 4 minutes. Season with salt and pepper. (Can be prepared 2 days ahead. Cover and chill.) Serve warm or at room temperature.""]",23.0,"""2004-08-20T04:00:00.000Z""","[""Food Processor"", ""Onion"", … ""Bon Appétit""]",403.0,"""This uses the same ingredients found in boudin blanc, the classic French white sausage. Start two da…",18.0,4.375,"""Boudin Blanc Terrine with Red Onion Confit ""","[""1 1/2 cups whipping cream"", ""2 medium onions, chopped"", … ""1/2 teaspoon sugar""]",1439.0
"[""In a large heavy saucepan cook diced fennel and onion in butter over moderate heat, stirring, until softened, about 10 minutes. Peel and cube potatoes. Add potatoes and broth to fennel mixture and simmer, covered, until potatoes are very tender, about 20 minutes. In a blender or food processor purée mixture in batches until smooth and return to saucepan. Stir in milk and salt and pepper to taste and simmer soup, stirring occasionally, 10 minutes, or until heated through."", ""Garnish soup with reserved fennel leaves.""]",7.0,"""2004-08-20T04:00:00.000Z""","[""Soup/Stew"", ""Dairy"", … ""New York""]",165.0,null,6.0,3.75,"""Potato and Fennel Soup Hodge ""","[""1 fennel bulb (sometimes called anise), stalks discarded, bulb cut into 1/2-inch dice, and feathery leaves reserved for garnish"", ""1 onion, diced"", … ""1 1/2 cups milk""]",165.0
"[""Heat oil in heavy large skillet over medium-high heat. Add onion; sauté until translucent and beginning to brown, about 4 minutes. Add wine and anchovy paste. Boil until reduced to 3/4 cup, about 3 minutes. Add tomatoes with juice; bring to boil."", ""Sprinkle fish with salt and pepper. Add fish to skillet atop tomato mixture. Reduce heat to low, cover, and simmer until fish is cooked through, about 9 minutes. Using slotted metal spatula, transfer fish to plate and tent with foil to keep warm. Mix olives, 2 t

In [11]:
df_json.columns

['directions',
 'fatGrams',
 'date',
 'categories',
 'calories',
 'desc',
 'proteinGrams',
 'rating',
 'recipeTitle',
 'ingredients',
 'sodiumMg']

In [12]:
# Find unique titles in both dataframes
common_titles = recipes["recipeTitle"].is_in(df_json["recipeTitle"]).sum()

total_titles = recipes["recipeTitle"].n_unique()  # Número total de valores únicos em df1
match_percentage = (common_titles / total_titles) * 100

print(f"Porcentagem de correspondência: {match_percentage:.2f}%")

Porcentagem de correspondência: 113.03%


In [13]:
# Concat both dataframes for creat a unique encode
df_combined = pl.concat([recipes, df_json], how="diagonal")  # Preenche valores ausentes com None


# Label Encoding é útil para situações onde os valores numéricos podem ser diretamente interpretados como categorias (ex.: IDs).
# encoding das receitas
df_combined = df_combined.with_columns(
    pl.col("recipeTitle")
    .rank("dense")  # Gera IDs únicos baseados nos valores
    .alias("recipeTitleEncoded")  # Nova coluna com encoding
)
df_combined.head()

recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,directions,date,categories,desc,ingredients,recipeTitleEncoded
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],str,list[str],str,list[str],u32
"""Lentil, Apple, and Turkey Wrap """,2.5,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,8805
"""Boudin Blanc Terrine with Red Onion Confit """,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,1743
"""Potato and Fennel Soup Hodge """,3.75,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,11930
"""Mahi-Mahi in Tomato Olive Sauce """,5.0,null,null,null,null,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,null,null,null,null,null,9054
"""Spinach Noodle Casserole """,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,null,null,null,null,null,15266


In [15]:
# In theses, we have duplicated values for each recipe.
df_combined.filter(pl.col("recipeTitleEncoded") == 15266)


recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,directions,date,categories,desc,ingredients,recipeTitleEncoded
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],str,list[str],str,list[str],u32
"""Spinach Noodle Casserole """,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,null,null,null,null,null,15266
"""Spinach Noodle Casserole """,3.125,547.0,20.0,32.0,452.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Preheat oven to 350°F. Lightly grease 8x8x2-inch glass baking dish. Blend spinach, noodles, sour cream, pesto sauce and nutmeg in large bowl. Spoon mixture into prepared dish. Sprinkle cheese over. Bake until set, about 45 minutes. Let stand 10 minutes.""]","""2004-08-20T04:00:00.000Z""","[""Cheese"", ""Dairy"", … ""California""]",null,"[""1 12-ounce package frozen spinach soufflé, thawed"", ""1/2 pound extra-wide egg noodles, freshly cooked"", … ""1 cup grated sharp cheddar cheese""]",15266


In [16]:
df_combined.describe()

statistic,recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,directions,date,categories,desc,ingredients,recipeTitleEncoded
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64
"""count""","""40163""",40152.0,31911.0,31819.0,31777.0,31907.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20052.0,20111.0,"""20111""",20111.0,"""13495""",20111.0,40163.0
"""null_count""","""19""",30.0,8271.0,8363.0,8405.0,8275.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20130.0,20071.0,"""20071""",20071.0,"""26687""",20071.0,19.0
"""mean""",null,3.713763,6315.397731,100.053364,346.487019,6218.715047,0.000848,0.001346,0.000349,0.005436,0.074606,0.159884,0.088171,0.000249,0.134899,0.018751,0.046679,0.028127,0.117245,0.052314,0.018103,0.022192,0.04633,0.0194,0.000698,0.067774,0.014213,0.156194,0.062388,0.04628,0.267903,0.003242,0.005536,0.403351,0.09231,0.341412,null,null,null,null,null,8896.950601
"""std""",null,1.341971,358809.73773,3837.905123,20443.230697,333098.795199,0.029105,0.036671,0.018681,0.07353,0.262761,0.366508,0.28355,0.015789,0.341624,0.135649,0.210955,0.165339,0.32172,0.222665,0.133327,0.147312,0.210203,0.137928,0.026415,0.251364,0.118371,0.363048,0.241865,0.210095,0.442878,0.056844,0.074197,0.490582,0.28947,0.474195,null,null,null,null,null,5145.622393
"""min""","""""Adult"" Pimiento Cheese """,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,"""1996-09-01T20:47:00.000Z""",null,"""""A blend of rums, exotic juices, liqueurs, aromatic spices, absinthe, and trepidation. One from the …",null,1.0
"""25%""",null,3.75,198.0,3.0,7.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,4415.0
"""50%""",null,4.375,331.0,8.0,17.0,294.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,8873.0
"""75%""",null,4.375,586.0,27.0,33.0,711.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,null,null,null,null,null,13356.0
"""max""","""Zuppa di Cavolo Nero, Cannellini, e Salsicce: Kale, White Bean, and Sausage Soup """,5.0,3.0111218e7,236489.0,1.722763e6,2.767511e7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,"""2016-12-13T13:00:00.000Z""",null,"""この味は驚くばかりである!""",null,17780.0


In [17]:
df_combined.columns

['recipeTitle',
 'rating',
 'calories',
 'proteinGrams',
 'fatGrams',
 'sodiumMg',
 'twentyTwoMinuteMeals',
 'threeIngredientRecipes',
 'thirtyDaysOfGroceries',
 'advancePrepRequired',
 'dairy',
 'dairyFree',
 'egg',
 'digestif',
 'dinner',
 'fatFree',
 'fish',
 'grill',
 'healthy',
 'lowCal',
 'lowCarb',
 'lowCholesterol',
 'lowFat',
 'lowSodium',
 'lowSugar',
 'lunch',
 'meat',
 'zeroSugar',
 'noCook',
 'pork',
 'quickEasy',
 'quickHealthy',
 'soy',
 'soyFree',
 'vegan',
 'vegetarian',
 'directions',
 'date',
 'categories',
 'desc',
 'ingredients',
 'recipeTitleEncoded']

In [ ]:
# Identificar as colunas para processamento, excluindo 'recipeTitleEncoded' e 'recipeTitle'
columns_to_process = [col for col in df_combined.columns if col not in ['recipeTitleEncoded', 'recipeTitle']]

# Construir o DataFrame preenchido
agg_operations = [
    pl.col("recipeTitle").first().alias("recipeTitle")
]

# Adicionar operações de preenchimento para todas as outras colunas dinamicamente
agg_operations.extend([
    pl.col(column_name).fill_null(strategy="forward")
                      .fill_null(strategy="backward")
                      .first().alias(column_name)
    for column_name in columns_to_process
])

df_filled = (
    df_combined
    .group_by("recipeTitleEncoded", maintain_order=True)
    .agg(agg_operations)
)



df_filled.filter(pl.col("recipeTitleEncoded") == 15268)

recipeTitleEncoded,recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,directions,date,categories,desc,ingredients
u32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],str,list[str],str,list[str]
15268,"""Spinach Porcini Stuffing """,4.375,214.0,10.0,11.0,376.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"[""Place porcini in small bowl; pour 1 cup boiling water over. Let stand until soft, about 45 minutes. Drain, reserving soaking liquid. Chop porcini; set aside."", ""Heat large nonstick skillet over medium heat. Add sausage; sauté until brown, breaking up into small pieces with back of fork. Add porcini, shallots, and garlic. Reduce heat to medium-low; cover and cook until sausage is cooked through, stirring occasionally, about 5 minutes. Transfer sausage mixture and any juices to medium bowl. Stir bread cubes, spinach, and rosemary into sausage mixture. Season with coarse salt and pepper. Mix in 1/4 cup reserved porcini soaking liquid. Cover and chill stuffing overnight."", … ""Available in the produce section of many supermarkets and at specialty foods stores and Italian markets.""]","""2008-11-04T04:00:00.000Z""","[""Mushroom"", ""Side"", … ""Soy Free""]",null,"[""1 1/2-ounce package dried porcini mushrooms*"", ""1 cup boiling water"", … ""1 large egg""]"


In [28]:
df_filled.describe()

statistic,recipeTitleEncoded,recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,directions,date,categories,desc,ingredients
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64
"""count""",17780.0,"""17780""",17771.0,14134.0,14110.0,14092.0,14133.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17736.0,17775.0,"""17775""",17775.0,"""11997""",17775.0
"""null_count""",1.0,"""1""",10.0,3647.0,3671.0,3689.0,3648.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,6.0,"""6""",6.0,"""5784""",6.0
"""mean""",8890.5,null,3.722919,6786.144191,94.350461,372.232756,6736.134013,0.000959,0.001522,0.000395,0.005638,0.075327,0.160916,0.090156,0.000169,0.142986,0.016858,0.049222,0.028586,0.12173,0.052718,0.018437,0.021425,0.046854,0.01917,0.000507,0.072395,0.014941,0.15742,0.057059,0.049166,0.266576,0.003608,0.005864,0.410465,0.086547,0.339817,null,null,null,null,null
"""std""",5132.788229,null,1.321637,379628.527194,3556.251275,21627.241169,352923.008995,0.030946,0.038988,0.019863,0.074878,0.263926,0.367463,0.286413,0.013005,0.350068,0.128744,0.216337,0.166644,0.326983,0.223476,0.134529,0.144802,0.211332,0.137126,0.022521,0.259148,0.121322,0.364207,0.231962,0.21622,0.442181,0.059964,0.076353,0.491932,0.281178,0.47366,null,null,null,null,null
"""min""",1.0,"""""Adult"" Pimiento Cheese """,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,"""1996-09-01T20:47:00.000Z""",null,"""""A blend of rums, exotic juices, liqueurs, aromatic spices, absinthe, and trepidation. One from the …",null
"""25%""",4446.0,null,3.75,208.0,3.0,8.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null
"""50%""",8891.0,null,4.375,351.0,9.0,18.0,310.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null
"""75%""",13335.0,null,4.375,605.0,29.0,35.0,740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,null,null,null,null,null
"""max""",17780.0,"""Zuppa di Cavolo Nero, Cannellini, e Salsicce: Kale, White Bean, and Sausage Soup """,5.0,3.0111218e7,236489.0,1.722763e6,2.767511e7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,"""2016-12-13T13:00:00.000Z""",null,"""この味は驚くばかりである!""",null


In [29]:
null_counts = df_filled.select(
    [pl.col(col).is_null().sum().alias(col) for col in recipes.columns]
)

print(null_counts)

shape: (1, 36)
┌─────────────┬────────┬──────────┬──────────────┬───┬─────┬─────────┬───────┬────────────┐
│ recipeTitle ┆ rating ┆ calories ┆ proteinGrams ┆ … ┆ soy ┆ soyFree ┆ vegan ┆ vegetarian │
│ ---         ┆ ---    ┆ ---      ┆ ---          ┆   ┆ --- ┆ ---     ┆ ---   ┆ ---        │
│ u32         ┆ u32    ┆ u32      ┆ u32          ┆   ┆ u32 ┆ u32     ┆ u32   ┆ u32        │
╞═════════════╪════════╪══════════╪══════════════╪═══╪═════╪═════════╪═══════╪════════════╡
│ 1           ┆ 10     ┆ 3647     ┆ 3671         ┆ … ┆ 45  ┆ 45      ┆ 45    ┆ 45         │
└─────────────┴────────┴──────────┴──────────────┴───┴─────┴─────────┴───────┴────────────┘


In [30]:
df_filled.head()

recipeTitleEncoded,recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,directions,date,categories,desc,ingredients
u32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],str,list[str],str,list[str]
8805,"""Lentil, Apple, and Turkey Wrap """,2.5,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[""1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool."", ""2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper."", ""3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.""]","""2006-09-01T04:00:00.000Z""","[""Sandwich"", ""Bean"", … ""Cookie""]",null,"[""4 cups low-sodium vegetable or chicken stock"", ""1 cup dried brown lentils"", … ""1/2 head Bibb lettuce""]"
1743,"""Boudin Blanc Terrine with Red Onion Confit """,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[""Combine first 9 ingredients in heavy medium saucepan. Add 3 shallots. Bring to simmer. Remove from heat, cover and let stand 30 minutes. Chill overnight."", ""Preheat oven to 325°F. Line 7-cup pâté or bread pan with plastic wrap. Melt butter in heavy small skillet over low heat. Add remaining 5 shallots. Cover and cook until very soft, stirring occasionally, about 15 minutes. Transfer to processor. Add pork, eggs, flour and Port and puree. Strain cream mixture, pressing on solids to extract as much liquid as possible. With processor running, add cream through feed tube and process just until combined with pork. Transfer to large bowl. Mix in currants."", … ""Heat oil in heavy large skillet over medium-high heat. Add onions and sauté until crisp-tender, about 8 minutes. Add all remaining ingredients and stir until reduced to thick glaze, about 4 minutes. Season with salt and pepper. (Can be prepared 2 days ahead. Cover and chill.) Serve warm or at room temperature.""]","""2004-08-20T04:00:00.000Z""","[""Food Processor"", ""Onion"", … ""Bon Appétit""]","""This uses the same ingredients found in boudin blanc, the classic French white sausage. Start two da…","[""1 1/2 cups whipping cream"", ""2 medium onions, chopped"", … ""1/2 teaspoon sugar""]"
11930,"""Potato and Fennel Soup Hodge """,3.75,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[""In a large heavy saucepan cook diced fennel and onion in butter over moderate heat, stirring, until softened, about 10 minutes. Peel and cube potatoes. Add potatoes and broth to fennel mixture and simmer, covered, until potatoes are very tender, about 20 minutes. In a blender or food processor purée mixture in batches until smooth and return to saucepan. Stir in milk and salt and pepper to taste and simmer soup, stirring occasionally, 10 minutes, or until heated through."", ""Garnish soup wi

In [15]:
df_filled.filter(pl.col("recipe_title_encoded") == 15268)['ingredients']


ingredients
list[str]
"[""1 1/2-ounce package dried porcini mushrooms*"", ""1 cup boiling water"", … ""1 large egg""]"


In [31]:
df_filled.columns

['recipeTitleEncoded',
 'recipeTitle',
 'rating',
 'calories',
 'proteinGrams',
 'fatGrams',
 'sodiumMg',
 'twentyTwoMinuteMeals',
 'threeIngredientRecipes',
 'thirtyDaysOfGroceries',
 'advancePrepRequired',
 'dairy',
 'dairyFree',
 'egg',
 'digestif',
 'dinner',
 'fatFree',
 'fish',
 'grill',
 'healthy',
 'lowCal',
 'lowCarb',
 'lowCholesterol',
 'lowFat',
 'lowSodium',
 'lowSugar',
 'lunch',
 'meat',
 'zeroSugar',
 'noCook',
 'pork',
 'quickEasy',
 'quickHealthy',
 'soy',
 'soyFree',
 'vegan',
 'vegetarian',
 'directions',
 'date',
 'categories',
 'desc',
 'ingredients']

## data igredientes

In [32]:
columns_ingredients = ['recipeTitleEncoded', 'recipeTitle', 'calories', 'ingredients']
df_ingredients = df_filled[columns_ingredients]

# Function to parse ingredients
def parse_ingredient(line):
    match = re.match(r"^([\d/]+)\s+(\w+)\s+(.+)$", line)
    if match:
        quantity = match.group(1)
        type_medida = match.group(2)
        ingredient = match.group(3)
        return quantity, type_medida, ingredient
    else:
        return None, None, line

# Explode the ingredients list, parse each line, and build the final DataFrame
df_ingredients = (
    df_ingredients.explode("ingredients")  # Expand the list of ingredients into individual rows
      .with_columns([
        pl.col("ingredients").map_elements(lambda x: parse_ingredient(x)[0]).alias("quantity"),
        pl.col("ingredients").map_elements(lambda x: parse_ingredient(x)[1]).alias("type"),
        pl.col("ingredients").map_elements(lambda x: parse_ingredient(x)[2]).alias("ingredient_for_recipe"),
    ])
    .drop("ingredients")  # Remove the original ingredients column if no longer needed
)
print(df_ingredients)



sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


shape: (180_508, 6)
┌────────────────────┬───────────────────────┬──────────┬──────────┬────────┬──────────────────────┐
│ recipeTitleEncoded ┆ recipeTitle           ┆ calories ┆ quantity ┆ type   ┆ ingredient_for_recip │
│ ---                ┆ ---                   ┆ ---      ┆ ---      ┆ ---    ┆ e                    │
│ u32                ┆ str                   ┆ f64      ┆ str      ┆ str    ┆ ---                  │
│                    ┆                       ┆          ┆          ┆        ┆ str                  │
╞════════════════════╪═══════════════════════╪══════════╪══════════╪════════╪══════════════════════╡
│ 8805               ┆ Lentil, Apple, and    ┆ 426.0    ┆ 4        ┆ cups   ┆ low-sodium vegetable │
│                    ┆ Turkey Wrap           ┆          ┆          ┆        ┆ or chicken stock     │
│ 8805               ┆ Lentil, Apple, and    ┆ 426.0    ┆ 1        ┆ cup    ┆ dried brown lentils  │
│                    ┆ Turkey Wrap           ┆          ┆          ┆   

## Data directions

In [ ]:
columns_directions = ['recipeTitleEncoded', 'recipeTitle', 'directions', 'desc']
df_directions = df_filled[columns_directions]
df_filled=  df_filled.drop("directions")
df_filled=  df_filled.drop("ingredients")
df_filled=  df_filled.drop("categories")
df_filled.head()

recipeTitleEncoded,recipeTitle,rating,calories,proteinGrams,fatGrams,sodiumMg,twentyTwoMinuteMeals,threeIngredientRecipes,thirtyDaysOfGroceries,advancePrepRequired,dairy,dairyFree,egg,digestif,dinner,fatFree,fish,grill,healthy,lowCal,lowCarb,lowCholesterol,lowFat,lowSodium,lowSugar,lunch,meat,zeroSugar,noCook,pork,quickEasy,quickHealthy,soy,soyFree,vegan,vegetarian,date,categories,desc
u32,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,list[str],str
8805,"""Lentil, Apple, and Turkey Wrap """,2.5,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""2006-09-01T04:00:00.000Z""","[""Sandwich"", ""Bean"", … ""Cookie""]",null
1743,"""Boudin Blanc Terrine with Red Onion Confit """,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"""2004-08-20T04:00:00.000Z""","[""Food Processor"", ""Onion"", … ""Bon Appétit""]","""This uses the same ingredients found in boudin blanc, the classic French white sausage. Start two da…"
11930,"""Potato and Fennel Soup Hodge """,3.75,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""2004-08-20T04:00:00.000Z""","[""Soup/Stew"", ""Dairy"", … ""New York""]",null
9054,"""Mahi-Mahi in Tomato Olive Sauce """,5.0,null,null,null,null,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"""2009-03-27T04:00:00.000Z""","[""Fish"", ""Olive"", … ""Kosher""]","""The Sicilian-style tomato sauce has tons of Mediterranean flavor, thanks to the orange peel, olives,…"
15266,"""Spinach Noodle Casserole """,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,"""2004-08-20T04:00:00.000Z""","[""Cheese"", ""Dairy"", … ""California""]",null


In [ ]:
df_directions = df_directions.fill_null("nao informado")

df_directions = df_directions.with_columns(
  pl.col("directions").list.join(" ").alias("directions")

)
df_directions.head()

recipeTitleEncoded,recipeTitle,directions,desc
u32,str,list[str],str
8805,"""Lentil, Apple, and Turkey Wrap ""","[""1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool."", ""2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper."", ""3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.""]",null
1743,"""Boudin Blanc Terrine with Red Onion Confit ""","[""Combine first 9 ingredients in heavy medium saucepan. Add 3 shallots. Bring to simmer. Remove from heat, cover and let stand 30 minutes. Chill overnight."", ""Preheat oven to 325°F. Line 7-cup pâté or bread pan with plastic wrap. Melt butter in heavy small skillet over low heat. Add remaining 5 shallots. Cover and cook until very soft, stirring occasionally, about 15 minutes. Transfer to processor. Add pork, eggs, flour and Port and puree. Strain cream mixture, pressing on solids to extract as much liquid as possible. With processor running, add cream through feed tube and process just until combined with pork. Transfer to large bowl. Mix in currants."", … ""Heat oil in heavy large skillet over medium-high heat. Add onions and sauté until crisp-tender, about 8 minutes. Add all remaining ingredients and stir until reduced to thick glaze, about 4 minutes. Season with salt and pepper. (Can be prepared 2 days ahead. Cover and chill.) Serve warm or at room temperature.""]","""This uses the same ingredients found in boudin blanc, the classic French white sausage. Start two da…"
11930,"""Potato and Fennel Soup Hodge ""","[""In a large heavy saucepan cook diced fennel and onion in butter over moderate heat, stirring, until softened, about 10 minutes. Peel and cube potatoes. Add potatoes and broth to fennel mixture and simmer, covered, until potatoes are very tender, about 20 minutes. In a blender or food processor purée mixture in batches until smooth and return to saucepan. Stir in milk and salt and pepper to taste and simmer soup, stirring occasionally, 10 minutes, or until heated through."", ""Garnish soup with reserved fennel leaves.""]",null
9054,"""Mahi-Mahi in Tomato Olive Sauce ""","[""Heat oil in heavy large skillet over medium-high heat. Add onion; sauté until translucent and beginning to brown, about 4 minutes. Add wine and anchovy paste. Boil until reduced to 3/4 cup, about 3 minutes. Add tomatoes with juice; bring to boil."", ""Sprinkle fish with salt and pepper. Add fish to skillet atop tomato mixture. Reduce heat to low, cover, and simmer until fish is cooked through, about 9 minutes. Using slotted metal spatula, transfer fish to plate and tent with foil to keep warm. Mix olives, 2 teaspoons oregano, and orange peel into sauce in skillet. Increase heat to high and boil until sauce is reduced and thickened, about 6 minutes. Season to taste with salt and pepper. Place 1 fish fillet on each of 4 plates. Pour sauce over and around fish, sprinkle with remaining 1 teaspoon oregano, and serve with warm toasted bread.""]","""The Sicilian-style tomato sauce has tons of Mediterranean flavor, thanks to the orange peel, olives,…"
15266,"""Spinach Noodle Casserole ""","[""Preheat oven to 350°F. Lightly grease 8x8x2-inch glass baking dish. Blend spinach, noodles, sour cream, pesto sauce and nutmeg in large bowl. Spoon mixture into prepared dish. Sprinkle cheese over. Bake until set, about 45 minutes. Let stand 1

# Conexão com o banco

In [37]:
import os
from sqlalchemy import create_engine, text
import yaml


In [38]:
docker_compose_path = r"C:\Users\fabia\Downloads\recipes_db\docker-compose.yml"
table_name = "food_good"

In [39]:
def get_db_config(docker_compose_path, db_service_name='postgres'):
    """
    Lê o arquivo docker-compose.yml e obtém as configurações de banco de dados.

    :param docker_compose_path: Caminho para o arquivo docker-compose.yml.
    :param db_service_name: Nome do serviço no arquivo docker-compose.yml.
    :return: Dicionário com as configurações do banco.
    """
    with open(docker_compose_path, 'r') as file:
        docker_compose = yaml.safe_load(file)

    db_service = docker_compose['services'][db_service_name]
    db_env = db_service.get('environment', {})

    return {
        'user': os.getenv('POSTGRES_USER', db_env.get('POSTGRES_USER', '')),
        'password': os.getenv('POSTGRES_PASSWORD', db_env.get('POSTGRES_PASSWORD', '')),
        'host': 'localhost',  # Para docker local, 'localhost' é o host
        'port': db_service['ports'][0].split(':')[0],  # Porta externa mapeada
        'database': os.getenv('POSTGRES_DB', db_env.get('POSTGRES_DB', 'good_food'))
    }

In [81]:
def get_db_connection_from_docker_compose(docker_compose_path, service_name="postgres"):
    """
    Extrai as informações do serviço do PostgreSQL a partir do arquivo docker-compose.yml
    e gera a string de conexão.
    
    :param docker_compose_path: Caminho para o arquivo docker-compose.yml.
    :param service_name: Nome do serviço no arquivo docker-compose.yml.
    :return: String de conexão para o banco de dados PostgreSQL.
    """
    # Ler o arquivo docker-compose.yml
    with open(docker_compose_path, "r") as file:
        docker_compose = yaml.safe_load(file)
    
    # Obter as informações do serviço
    db_service = docker_compose["services"].get(service_name)
    if not db_service:
        raise ValueError(f"Serviço '{service_name}' não encontrado no arquivo docker-compose.yml.")
    
    # Extrair as variáveis de ambiente do serviço
    db_env = db_service.get("environment", {})
    user = db_env.get("POSTGRES_USER", "postgres")
    password = db_env.get("POSTGRES_PASSWORD", "")
    database = db_env.get("POSTGRES_DB", "postgres")
    host = "localhost"  # Padrão para conexões locais
    port = db_service.get("ports", ["5432:5432"])[0].split(":")[0]

    # Criar a string de conexão
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    return connection_string

docker_compose_path = r"C:\Users\fabia\Downloads\recipes_db\docker-compose.yml"
table_name = "food_good"

connection = get_db_connection_from_docker_compose(docker_compose_path)

data_to_insert = df_directions.write_database(
    table_name="directions_02",
    connection=connection,
    engine="sqlalchemy",
    if_table_exists="replace"
)  
# connection

# db_config = get_db_config(docker_compose_path)
# # Configuração da conexão com o banco
# engine = create_engine(
#     f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
# )


In [46]:
# Inspecionar o tipo de cada coluna
print(df_directions.schema)

# # Verificar o tipo de dados para cada linha em uma coluna específica
# for col in df_filled.columns:
#     print(f"Coluna: {col}")
#     print(df_filled[col].to_list())

Schema([('recipeTitleEncoded', UInt32), ('recipeTitle', String), ('directions', List(String)), ('desc', String)])


781